<a href="https://colab.research.google.com/github/Hacxmr/Cyber_Threat_Detection/blob/main/Avishkar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow


In [3]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, LSTM, Bidirectional, Reshape
from tensorflow.keras.utils import to_categorical


import pandas as pd
import os

# Step 1: Load feature names (excluding header rows if any)
feature_meta = pd.read_csv("/content/UNSW_NB15/NUSW-NB15_features.csv", encoding='latin1')

# Step 2: Extract column names and append 'attack_cat' and 'label'
columns = feature_meta['Name'].tolist() + ['attack_cat', 'label']

# Step 3: Deduplicate column names by appending suffixes to duplicates
def deduplicate(names):
    seen = {}
    deduped = []
    for name in names:
        if name not in seen:
            seen[name] = 1
            deduped.append(name)
        else:
            seen[name] += 1
            deduped.append(f"{name}_{seen[name]}")
    return deduped

columns = deduplicate(columns)

# Step 4: Load the 4 parts of UNSW-NB15 dataset
folder = "/content/UNSW_NB15"
files = ["UNSW-NB15_1.csv", "UNSW-NB15_2.csv", "UNSW-NB15_3.csv", "UNSW-NB15_4.csv"]

df = pd.concat([
    pd.read_csv(os.path.join(folder, file), header=None, names=columns, low_memory=False)
    for file in files
], ignore_index=True)

# Step 5: Show shape and preview
print("Shape of dataset:", df.shape)
df.head()


Shape of dataset: (2540047, 51)


,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label,attack_cat_2,label
0,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,...,7,1,3,1,1,1,NaN,0,NaN,NaN
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,4,2,3,1,1,2,NaN,0,NaN,NaN
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,8,1,2,2,1,1,NaN,0,NaN,NaN
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,9,1,1,1,1,1,NaN,0,NaN,NaN
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,9,1,1,1,1,1,NaN,0,NaN,NaN


In [4]:
# === Encode categorical columns ===
cat_cols = df.select_dtypes(include=['object']).columns
for col in cat_cols:
    df[col] = df[col].astype(str)
    df[col] = LabelEncoder().fit_transform(df[col])

# === Features and target ===
X = df.drop(['label', 'attack_cat'], axis=1)
y = df['label']
y_cat = to_categorical(y)

# === Normalize features ===
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# === Train-test split ===
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_cat, test_size=0.2, random_state=42)

# === Reshape for CNN/LSTM ===
X_train_seq = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_seq = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# === Define model builders ===

def build_dnn(input_shape):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_shape,)),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(2, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def build_cnn(input_shape):
    model = Sequential([
        Conv1D(64, 3, activation='relu', input_shape=input_shape),
        MaxPooling1D(2),
        Dropout(0.3),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(2, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def build_lstm(input_shape):
    model = Sequential([
        LSTM(64, input_shape=input_shape),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(2, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def build_bilstm(input_shape):
    model = Sequential([
        Bidirectional(LSTM(64), input_shape=input_shape),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(2, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# === Train and evaluate models ===
models = {
    'DNN': build_dnn(X_train.shape[1]),
    'CNN': build_cnn((X_train.shape[1], 1)),
    'LSTM': build_lstm((X_train.shape[1], 1)),
    'BiLSTM': build_bilstm((X_train.shape[1], 1)),
}

histories = {}

for name, model in models.items():
    print(f"\nTraining {name} model...")
    if name == 'DNN':
        history = model.fit(X_train, y_train, epochs=5, batch_size=512, validation_data=(X_test, y_test))
    else:
        history = model.fit(X_train_seq, y_train, epochs=5, batch_size=512, validation_data=(X_test_seq, y_test))
    histories[name] = history
    _, acc = model.evaluate(X_test_seq if name != 'DNN' else X_test, y_test)
    print(f"{name} Accuracy: {acc:.4f}")

/usr/local/lib/python3.11/dist-packages/pandas/core/series.py:1031: RuntimeWarning: invalid value encountered in cast
  arr = np.asarray(values, dtype=dtype)


ValueError: negative dimensions are not allowed

In [15]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, LSTM, Reshape
from sklearn.metrics import classification_report

# === Step 1: Load UNSW-NB15 Dataset ===

folder = "/content/UNSW_NB15"  # path to folder containing CSVs
files = ["UNSW-NB15_1.csv", "UNSW-NB15_2.csv", "UNSW-NB15_3.csv", "UNSW-NB15_4.csv"]
col_file = os.path.join(folder, "/content/UNSW_NB15/NUSW-NB15_features.csv")

# Read column names from UNSW-NB15_features.csv
col_df = pd.read_csv(col_file, encoding='latin1')
columns = col_df['Name'].tolist()


# Deduplicate columns (some are repeated)
_, idx = np.unique(columns, return_index=True)
columns = [columns[i] for i in sorted(idx)]

# Load all parts
df_list = []
for file in files:
    df_part = pd.read_csv(os.path.join(folder, file), header=None, low_memory=False)
    df_part.columns = columns
    df_list.append(df_part)

df = pd.concat(df_list, ignore_index=True)

# === Step 2: Preprocessing ===

# Drop non-numeric or categorical columns that can't be directly used
df.drop(['id', 'attack_cat'], axis=1, errors='ignore', inplace=True)

# Encode categorical columns
cat_cols = df.select_dtypes(include=['object']).columns
for col in cat_cols:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))

# Drop rows with missing values
df = df.dropna()

# Separate features and labels
X = df.drop('Label', axis=1)
y = df['Label']

# One-hot encode the label (binary classification)
y_cat = to_categorical(y)

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# === Step 3: Train/Test Split ===
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_cat, test_size=0.2, random_state=42)

# === Step 4: Define Deep Learning Models ===

def build_mlp(input_shape):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_shape,)),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(2, activation='softmax')
    ])
    return model

def build_cnn(input_shape):
    model = Sequential([
        Reshape((input_shape, 1), input_shape=(input_shape,)),
        Conv1D(64, 3, activation='relu'),
        MaxPooling1D(2),
        Dropout(0.3),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(2, activation='softmax')
    ])
    return model

def build_lstm(input_shape):
    model = Sequential([
        Reshape((input_shape, 1), input_shape=(input_shape,)),
        LSTM(64, return_sequences=False),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dense(2, activation='softmax')
    ])
    return model

# === Step 5: Train and Evaluate Models ===

def train_evaluate_model(model, name):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    print(f"\nTraining {name}...")
    model.fit(X_train, y_train, epochs=5, batch_size=512, validation_split=0.2, verbose=1)
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y_test, axis=1)
    print(f"\n{name} Classification Report:\n")
    print(classification_report(y_true, y_pred_classes))

# Train all models
mlp_model = build_mlp(X_train.shape[1])
cnn_model = build_cnn(X_train.shape[1])
lstm_model = build_lstm(X_train.shape[1])

train_evaluate_model(mlp_model, "MLP")
train_evaluate_model(cnn_model, "CNN")
train_evaluate_model(lstm_model, "LSTM")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Training MLP...
Epoch 1/5
1360/1360 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.9928 - loss: 0.0313 - val_accuracy: 0.9968 - val_loss: 0.0064
Epoch 2/5
1360/1360 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.9968 - loss: 0.0067 - val_accuracy: 0.9970 - val_loss: 0.0058
Epoch 3/5
1360/1360 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.9970 - loss: 0.0062 - val_accuracy: 0.9973 - val_loss: 0.0055
Epoch 4/5
1360/1360 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9972 - loss: 0.0054 - val_accuracy: 0.9974 - val_loss: 0.0055
Epoch 5/5
1360/1360 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9972 - loss: 0.0054 - val_accuracy: 0.9973 - val_loss: 0.0054
6796/6796 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step

MLP Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    212955
           1       0.93      0.94      0.94      4486

    accuracy                           1.00    217441
   macro avg       0.97      0.97      0.97  

In [16]:
import pandas as pd
from tabulate import tabulate

# Create a DataFrame with model performance
data = {
    "Metric": ["Accuracy", "Precision (Attack)", "Recall (Attack)", "F1-Score (Attack)", "Val Accuracy (Last Epoch)", "Training Time (5 Epochs)"],
    "MLP": ["100%", "0.93", "0.94", "0.94", "99.73%", "~50 sec"],
    "CNN": ["100%", "0.92", "0.95", "0.94", "99.74%", "~7 min"],
    "LSTM": ["100%", "0.89", "0.94", "0.92", "99.64%", "~22 min"]
}

df = pd.DataFrame(data)

# Display the table in a pretty format using tabulate
print(tabulate(df, headers='keys', tablefmt='fancy_grid', showindex=False))


╒═══════════════════════════╤═════════╤════════╤═════════╕
│ Metric                    │ MLP     │ CNN    │ LSTM    │
╞═══════════════════════════╪═════════╪════════╪═════════╡
│ Accuracy                  │ 100%    │ 100%   │ 100%    │
├───────────────────────────┼─────────┼────────┼─────────┤
│ Precision (Attack)        │ 0.93    │ 0.92   │ 0.89    │
├───────────────────────────┼─────────┼────────┼─────────┤
│ Recall (Attack)           │ 0.94    │ 0.95   │ 0.94    │
├───────────────────────────┼─────────┼────────┼─────────┤
│ F1-Score (Attack)         │ 0.94    │ 0.94   │ 0.92    │
├───────────────────────────┼─────────┼────────┼─────────┤
│ Val Accuracy (Last Epoch) │ 99.73%  │ 99.74% │ 99.64%  │
├───────────────────────────┼─────────┼────────┼─────────┤
│ Training Time (5 Epochs)  │ ~50 sec │ ~7 min │ ~22 min │
╘═══════════════════════════╧═════════╧════════╧═════════╛


In [14]:
print(df.columns.tolist())


['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur', 'sbytes', 'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'service', 'Sload', 'Dload', 'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz', 'dmeansz', 'trans_depth', 'res_bdy_len', 'Sjit', 'Djit', 'Stime', 'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt', 'synack', 'ackdat', 'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'Label']
